# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [17]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain_community.llms import OpenAI

In [13]:
from pathlib import Path

file = Path() / ".." / ".." /"data" / "OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file, encoding="utf8")

In [7]:
from langchain.indexes.vectorstore import VectorstoreIndexCreator

In [14]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

C:\Users\sayantanchakraborty\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [15]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [18]:
llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

response = index.query(query, 
                       llm = llm_replacement_model)

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [0.00325083..., -0.02111668333220731]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [0.00325083..., -0.02111668333220731]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [19]:
display(Markdown(response))

NameError: name 'response' is not defined

## Step By Step

In [22]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file, encoding="utf8")

In [23]:
docs = loader.load()

In [24]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': WindowsPath('../../data/OutdoorClothingCatalog_1000.csv'), 'row': 0})

In [25]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [26]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [27]:
print(len(embed))

1536


In [28]:
print(embed[:5])

[-0.0219938028518557, 0.006747527976699378, -0.018252847709138532, -0.03916704653175717, -0.013997197145759583]


In [29]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for text-embedding-ada-002 in organization org-p07ChwGMGF0UvE8WPsNiEKYN on tokens per min (TPM): Limit 150000, Requested 176510. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

Reminder: Download your notebook to you local computer to save your work.